# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [22]:
import torch.optim as optim
import torch.nn.functional as F

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        num_classes = 10
        self.feature = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, num_classes)
        )
    def forward(self,x):
        x = self.feature(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        #x = F.log_softmax(x,dim = 1)
        return x
    
model = SimpleNet()

#TODO:define loss function and optimiter
criterion =  nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

Next, we can start to train and evaluate!

In [32]:
# train and evaluate
def eval_(model, epoch, eval_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (feature, labels) in enumerate(eval_loader):
            feature, labels = feature.to(device, dtype=torch.float), labels.to(device, dtype=torch.long)
            output = model(feature)
            test_loss += nn.CrossEntropyLoss()(output, labels).item()
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= (batch_idx+1)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(eval_loader.dataset),
        100. * correct / len(eval_loader.dataset)))
    return correct/len(eval_loader.dataset)

In [34]:
use_cuda=True
device = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)
model.train()
for epoch in range(NUM_EPOCHS):
    if epoch==1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.005
    if epoch==3:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001
    for  batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device, dtype=torch.float), labels.to(device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(images)
        loss = nn.CrossEntropyLoss()(output, labels)
        loss.backward()
        optimizer.step()
        if batch_idx%100==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        # TODO:forward + backward + optimize
    eval_(model,NUM_EPOCHS, test_loader,device)
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
    
    
    


Train Epoch: 0 [0/60000 (0%)]	Loss: 0.047366
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.014706
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.004958
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.029174
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.027023

Test set: Average loss: 0.0152, Accuracy: 9934/10000 (99.340%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005335
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.004247
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.000430
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.001442
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.001059

Test set: Average loss: 0.0159, Accuracy: 9930/10000 (99.300%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.001724
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000317
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.000357
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.000288
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.000912

Test set: Average loss: 0.0180, Accuracy: 9927/10000 (99.270%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000409
Train Epoch: 3 [12800/600

In [35]:
train_accuracy = eval_(model,NUM_EPOCHS, train_loader,device)
test_accuracy = eval_(model,NUM_EPOCHS, test_loader,device)



Test set: Average loss: 0.0006, Accuracy: 59902/60000 (99.837%)


Test set: Average loss: 0.0146, Accuracy: 9938/10000 (99.380%)



#### Q5:
Please print the training and testing accuracy.

In [36]:
print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))

Training accuracy: 99.84%
Testing accuracy: 99.38%
